# Quantium Data Analytics Virtual Experience
## Part 1: Data Preparation and Customer Analytics
We need to present a strategic recommendation to Julia that is supported by data which she can then use for the upcoming category review however to do so we need to analyse the data to understand the current purchasing trends and behaviours. The client is particularly interested in customer segments and their chip purchasing behaviour. Consider what metrics would help describe the customers’ purchasing behaviour.  

In [594]:
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2

In [611]:
try:
    db = psycopg2.connect("host=127.0.0.1 dbname=chips")
    cur = db.cursor()
    query = """
        drop table purchases;
        drop table transactions;

        CREATE TABLE purchases (
            LYLTY_CARD_NBR integer,
            LIFESTAGE text,
            PREMIUM_CUSTOMER text,
            PRIMARY KEY (LYLTY_CARD_NBR)
        );

        CREATE TABLE transactions (
            index integer,
            DATE integer,
            STORE_NBR integer,
            LYLTY_CARD_NBR integer,
            TXN_ID integer,
            PROD_NBR integer,
            PROD_NAME text,
            PROD_QTY integer,
            TOT_SALES float,
            PRIMARY KEY (index)
        );"""
    cur.execute(query)
except psycopg2.Error as err:
    print("DB error: ", err)

In [612]:
with open('QVI_transaction_data.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'transactions', sep=',')
db.commit()

In [397]:
query = "select * from transactions;"
transact_df = sqlio.read_sql_query(query, db)

In [606]:
with open('QVI_purchase_behaviour.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'purchases', sep=',')
db.commit()

In [399]:
query = "select * from purchases;"
purchase_df = sqlio.read_sql_query(query, db)

In [400]:
purchase_df.head()

,lylty_card_nbr,lifestage,premium_customer
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [401]:
transact_df.head()

,index,date,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_name,prod_qty,tot_sales
0,0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [402]:
purchase_df.shape

(72637, 3)

In [403]:
transact_df.shape

(264836, 9)

In [404]:
purchase_df.dtypes

lylty_card_nbr       int64
lifestage           object
premium_customer    object
dtype: object

In [406]:
purchase_df["lylty_card_nbr"].astype("object")

0           1000
1           1002
2           1003
3           1004
4           1005
          ...   
72632    2370651
72633    2370701
72634    2370751
72635    2370961
72636    2373711
Name: lylty_card_nbr, Length: 72637, dtype: object

In [407]:
transact_df["lylty_card_nbr"].astype("object")

0           1000
1           1307
2           1343
3           2373
4           2426
           ...  
264831    272319
264832    272358
264833    272379
264834    272379
264835    272380
Name: lylty_card_nbr, Length: 264836, dtype: object

In [408]:
transact_df.dtypes

index               int64
date                int64
store_nbr           int64
lylty_card_nbr      int64
txn_id              int64
prod_nbr            int64
prod_name          object
prod_qty            int64
tot_sales         float64
dtype: object

In [409]:
transact_df.describe()

,index,date,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_qty,tot_sales
count,264836.000000,264836.000000,264836.00000,2.648360e+05,2.648360e+05,264836.000000,264836.000000,264836.000000
mean,132417.500000,43464.036260,135.08011,1.355495e+05,1.351583e+05,56.583157,1.907309,7.304200
std,76451.712283,105.389282,76.78418,8.057998e+04,7.813303e+04,32.826638,0.643654,3.083226
min,0.000000,43282.000000,1.00000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.500000
25%,66208.750000,43373.000000,70.00000,7.002100e+04,6.760150e+04,28.000000,2.000000,5.400000
50%,132417.500000,43464.000000,130.00000,1.303575e+05,1.351375e+05,56.000000,2.000000,7.400000
75%,198626.250000,43555.000000,203.00000,2.030942e+05,2.027012e+05,85.000000,2.000000,9.200000
max,264835.000000,43646.000000,272.00000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000


In [410]:
purchase_df.lifestage.value_counts()

RETIREES                  14805
OLDER SINGLES/COUPLES     14609
YOUNG SINGLES/COUPLES     14441
OLDER FAMILIES             9780
YOUNG FAMILIES             9178
MIDAGE SINGLES/COUPLES     7275
NEW FAMILIES               2549
Name: lifestage, dtype: int64

In [411]:
purchase_df["premium_customer"].value_counts()

Mainstream    29245
Budget        24470
Premium       18922
Name: premium_customer, dtype: int64

In [412]:
purchase_df["lylty_card_nbr"].unique

<bound method Series.unique of 0           1000
1           1002
2           1003
3           1004
4           1005
          ...   
72632    2370651
72633    2370701
72634    2370751
72635    2370961
72636    2373711
Name: lylty_card_nbr, Length: 72637, dtype: int64>

In [413]:
transact_df["date"].head()

0    43390
1    43599
2    43605
3    43329
4    43330
Name: date, dtype: int64

Converting Excel Date to proper format

In [415]:
import xlrd

transact_df["date"] = transact_df["date"].apply(lambda x: xlrd.xldate_as_datetime(x, 0))
transact_df["date"] = transact_df["date"].apply(lambda x: x.date())
transact_df["date"] = transact_df["date"].apply(lambda x: x.isoformat())
transact_df["date"]

0         2018-10-17
1         2019-05-14
2         2019-05-20
3         2018-08-17
4         2018-08-18
             ...    
264831    2019-03-09
264832    2018-08-13
264833    2018-11-06
264834    2018-12-27
264835    2018-09-22
Name: date, Length: 264836, dtype: object

In [416]:
transact_df["store_nbr"].value_counts()

226    2022
88     1873
93     1832
165    1819
237    1785
       ... 
11        2
31        2
206       2
76        1
92        1
Name: store_nbr, Length: 272, dtype: int64

In [417]:
transact_df.isnull().any()

index             False
date              False
store_nbr         False
lylty_card_nbr    False
txn_id            False
prod_nbr          False
prod_name         False
prod_qty          False
tot_sales         False
dtype: bool

In [418]:
purchase_df.isnull().any()

lylty_card_nbr      False
lifestage           False
premium_customer    False
dtype: bool

In [420]:
transact_df["prod_name"].value_counts()

Kettle Mozzarella   Basil & Pesto 175g      3304
Kettle Tortilla ChpsHny&Jlpno Chili 150g    3296
Cobs Popd Swt/Chlli &Sr/Cream Chips 110g    3269
Tyrrells Crisps     Ched & Chives 165g      3268
Cobs Popd Sea Salt  Chips 110g              3265
                                            ... 
RRD Pc Sea Salt     165g                    1431
Woolworths Medium   Salsa 300g              1430
NCC Sour Cream &    Garden Chives 175g      1419
French Fries Potato Chips 175g              1418
WW Crinkle Cut      Original 175g           1410
Name: prod_name, Length: 114, dtype: int64

In [421]:
transact_df["chip_name"] = transact_df["prod_name"].replace(r'[^a-zA-Z ]|g$', '', regex=True)
transact_df["chip_name"].replace(r'\s+', " ", regex=True, inplace=True)
transact_df["chip_name"] = transact_df["chip_name"].str.strip()

In [423]:
transact_df["chip_name"].value_counts()

Kettle Mozzarella Basil Pesto         3304
Kettle Tortilla ChpsHnyJlpno Chili    3296
Cobs Popd SwtChlli SrCream Chips      3269
Tyrrells Crisps Ched Chives           3268
Cobs Popd Sea Salt Chips              3265
                                      ... 
RRD Pc Sea Salt                       1431
Woolworths Medium Salsa               1430
NCC Sour Cream Garden Chives          1419
French Fries Potato Chips             1418
WW Crinkle Cut Original               1410
Name: chip_name, Length: 114, dtype: int64

In [425]:
trans_df_clean = transact_df.drop(transact_df.loc[transact_df["prod_name"].str.contains("Salsa") == True].index)

Outlier of 200 chips

In [427]:
trans_df_clean[trans_df_clean["prod_qty"] == 200]

,index,date,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_name,prod_qty,tot_sales,chip_name
69762,69762,2018-08-19,226,226000,226201,4,Dorito Corn Chp Supreme 380g,200,650.0,Dorito Corn Chp Supreme
69763,69763,2019-05-20,226,226000,226210,4,Dorito Corn Chp Supreme 380g,200,650.0,Dorito Corn Chp Supreme


In [428]:
trans_df_clean[trans_df_clean["lylty_card_nbr"] == 226000]

,index,date,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_name,prod_qty,tot_sales,chip_name
69762,69762,2018-08-19,226,226000,226201,4,Dorito Corn Chp Supreme 380g,200,650.0,Dorito Corn Chp Supreme
69763,69763,2019-05-20,226,226000,226210,4,Dorito Corn Chp Supreme 380g,200,650.0,Dorito Corn Chp Supreme


Only have two purchases, and both of 200 chips. Since we want to look at retail customers, we can drop this.

In [429]:
trans_df_clean.drop(trans_df_clean[trans_df_clean["lylty_card_nbr"] == 226000].index, inplace=True)

In [430]:
trans_df_clean.describe()

,index,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_qty,tot_sales
count,246740.000000,246740.000000,2.467400e+05,2.467400e+05,246740.000000,246740.000000,246740.000000
mean,132435.834745,135.050361,1.355303e+05,1.351304e+05,56.352213,1.906456,7.316113
std,76465.694879,76.786971,8.071520e+04,7.814760e+04,33.695235,0.342499,2.474897
min,0.000000,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,66268.750000,70.000000,7.001500e+04,6.756875e+04,26.000000,2.000000,5.800000
50%,132445.500000,130.000000,1.303670e+05,1.351815e+05,53.000000,2.000000,7.400000
75%,198627.250000,203.000000,2.030832e+05,2.026522e+05,87.000000,2.000000,8.800000
max,264835.000000,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000


In [432]:
trans_df_clean.date.value_counts().sort_values()

2019-06-13    607
2018-09-22    609
2018-11-25    610
2018-10-18    611
2019-06-24    612
             ... 
2018-12-20    808
2018-12-19    839
2018-12-22    840
2018-12-23    853
2018-12-24    865
Name: date, Length: 364, dtype: int64

We see that we are missing a day of data - and it looks like it is Christmas day. Since shops are not usually open, we can assume that there are no sales.

In [433]:
trans_df_clean["chip_size"] = trans_df_clean["prod_name"].str.extract(pat = '(\d+)').astype('int')

In [434]:
trans_df_clean.describe()

,index,store_nbr,lylty_card_nbr,txn_id,prod_nbr,prod_qty,tot_sales,chip_size
count,246740.000000,246740.000000,2.467400e+05,2.467400e+05,246740.000000,246740.000000,246740.000000,246740.000000
mean,132435.834745,135.050361,1.355303e+05,1.351304e+05,56.352213,1.906456,7.316113,175.583521
std,76465.694879,76.786971,8.071520e+04,7.814760e+04,33.695235,0.342499,2.474897,59.432118
min,0.000000,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000,70.000000
25%,66268.750000,70.000000,7.001500e+04,6.756875e+04,26.000000,2.000000,5.800000,150.000000
50%,132445.500000,130.000000,1.303670e+05,1.351815e+05,53.000000,2.000000,7.400000,170.000000
75%,198627.250000,203.000000,2.030832e+05,2.026522e+05,87.000000,2.000000,8.800000,175.000000
max,264835.000000,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000,380.000000


In [435]:
trans_df_clean["brand"] = trans_df_clean["chip_name"].apply(lambda x: x.split(" ")[0])

In [436]:
trans_df_clean["brand"].value_counts()

Kettle        41288
Smiths        27390
Pringles      25102
Doritos       22041
Thins         14075
RRD           11894
Infuzions     11057
WW            10320
Cobs           9693
Tostitos       9471
Twisties       9454
Tyrrells       6442
Grain          6272
Natural        6050
Cheezels       4603
CCs            4551
Red            4427
Dorito         3183
Infzns         3144
Smith          2963
Cheetos        2927
Snbts          1576
Burger         1564
Woolworths     1516
GrnWves        1468
Sunbites       1432
NCC            1419
French         1418
Name: brand, dtype: int64

In [437]:
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("WW", "Woolworths")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("Red", "RRD")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("Infzns", "Infuzions")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace(r"Grain$", "GrainWaves", regex=True)
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("GrnWves", "GrainWaves")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace(r"Dorito$", "Doritos", regex=True)
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("Snbts", "Sunbites")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace("Natural", "NCC")
trans_df_clean["brand"] = trans_df_clean["brand"].str.replace(r"Smith$", "Smiths", regex=True)

In [439]:
trans_df_clean["brand"].value_counts()

Kettle        41288
Smiths        30353
Doritos       25224
Pringles      25102
RRD           16321
Infuzions     14201
Thins         14075
Woolworths    11836
Cobs           9693
Tostitos       9471
Twisties       9454
GrainWaves     7740
NCC            7469
Tyrrells       6442
Cheezels       4603
CCs            4551
Sunbites       3008
Cheetos        2927
Burger         1564
French         1418
Name: brand, dtype: int64

### We have now completed data cleaning and some creations of categories

In [440]:
combined_df = trans_df_clean.merge(purchase_df, how='left', on="lylty_card_nbr")

In [ ]:
combined_df.to_csv("analysis.csv")

In [441]:
combined_df.isnull().any()

index               False
date                False
store_nbr           False
lylty_card_nbr      False
txn_id              False
prod_nbr            False
prod_name           False
prod_qty            False
tot_sales           False
chip_name           False
chip_size           False
brand               False
lifestage           False
premium_customer    False
dtype: bool

In [451]:
combined_df.dtypes

index                 int64
date                 object
store_nbr             int64
lylty_card_nbr        int64
txn_id                int64
prod_nbr              int64
prod_name            object
prod_qty              int64
tot_sales           float64
chip_name            object
chip_size             int64
brand                object
lifestage            object
premium_customer     object
price               float64
dtype: object

In [442]:
import scipy
from scipy.stats import ttest_ind

In [452]:
combined_df['price'] = combined_df['tot_sales']/combined_df['prod_qty']
combined_df['price']

0         3.00
1         2.10
2         1.45
3         3.00
4         4.60
          ... 
246735    5.40
246736    4.40
246737    4.40
246738    3.90
246739    4.40
Name: price, Length: 246740, dtype: float64

In [575]:
mainstream_midage = combined_df[(combined_df['lifestage'] == 'MIDAGE SINGLES/COUPLES') & ((combined_df['premium_customer'] == 'Mainstream'))]
other = combined_df[(combined_df['lifestage'] == 'MIDAGE SINGLES/COUPLES') & (combined_df['premium_customer'] != 'Mainstream')]

ttest_ind(mainstream_midage['price'], other['price'], equal_var=False)

Ttest_indResult(statistic=16.864135225719995, pvalue=1.9622867289480684e-63)

In [576]:
segment = combined_df[(combined_df['lifestage'] == 'MIDAGE SINGLES/COUPLES') & (combined_df['premium_customer'] == 'Mainstream')]
other = combined_df[(combined_df['lifestage'] != 'MIDAGE SINGLES/COUPLES') & (combined_df['premium_customer'] != 'Mainstream')]

In [577]:
num_chips_segment = sum(segment['prod_qty'])
num_chips_other = sum(other['prod_qty'])

In [578]:
segment_num_chips_by_brand = pd.DataFrame(segment.groupby('brand').sum('prod_qty')['prod_qty']).reset_index()
other_num_chips_by_brand = pd.DataFrame(other.groupby('brand').sum('prod_qty')['prod_qty']).reset_index()

In [579]:
segment_num_chips_by_brand['target_segment'] = segment_num_chips_by_brand['prod_qty'] / num_chips_segment
other_num_chips_by_brand['other'] = other_num_chips_by_brand['prod_qty'] / num_chips_other

brand_prop = segment_num_chips_by_brand.merge(other_num_chips_by_brand, on='brand')
brand_prop.drop(columns=['prod_qty_x', 'prod_qty_y'], inplace=True)

In [580]:
brand_prop['brand_affinity'] = brand_prop['target_segment']/brand_prop['other']
brand_prop.sort_values(by='brand_affinity', ignore_index=True, ascending= False)

,brand,target_segment,other,brand_affinity
0,Cobs,0.044831,0.038361,1.168651
1,Kettle,0.192571,0.165381,1.164404
2,Twisties,0.043935,0.037806,1.162137
3,Tostitos,0.043558,0.038117,1.142742
4,Doritos,0.108895,0.100242,1.086328
5,Infuzions,0.061755,0.056921,1.084925
6,Tyrrells,0.026917,0.025577,1.052420
7,Cheezels,0.019846,0.018927,1.048560
8,Pringles,0.104181,0.100647,1.035114
9,GrainWaves,0.031962,0.030965,1.032194


In [582]:
segment_num_chips_by_size = pd.DataFrame(segment.groupby('chip_size').sum('prod_qty')['prod_qty']).reset_index()
other_num_chips_by_size = pd.DataFrame(other.groupby('chip_size').sum('prod_qty')['prod_qty']).reset_index()

In [584]:
segment_num_chips_by_size['target_segment'] = segment_num_chips_by_size['prod_qty'] / num_chips_segment
other_num_chips_by_size['other'] = other_num_chips_by_size['prod_qty'] / num_chips_other

brand_prop = segment_num_chips_by_size.merge(other_num_chips_by_size, on='chip_size')
brand_prop.drop(columns=['prod_qty_x', 'prod_qty_y'], inplace=True)

In [586]:
brand_prop['preferred_size'] = brand_prop['target_segment']/brand_prop['other']
brand_prop.sort_values(by='preferred_size', ignore_index=True, ascending=False)

,chip_size,target_segment,other,preferred_size
0,270,0.030736,0.024848,1.236940
1,330,0.059728,0.050744,1.177032
2,110,0.102060,0.088903,1.147997
3,135,0.014519,0.012901,1.125448
4,210,0.027719,0.024762,1.119413
5,380,0.028426,0.025423,1.118130
6,134,0.104181,0.100647,1.035114
7,250,0.013199,0.012957,1.018687
8,175,0.268562,0.271649,0.988634
9,150,0.160420,0.162967,0.984374


In [589]:
combined_df[combined_df['chip_size'] == 270]['brand'].unique()

array(['Twisties'], dtype=object)